# NGMS Champions - GeoVista Demonstration

---

## Regridding Visualisation

This section is mostly setting up demonstration objects - the regridding and visualisation is at the end of the section.

In [ ]:
from iris import load_cube
from iris.experimental.ugrid import PARSE_UGRID_ON_LOAD
from iris.tests import get_data_path
import numpy as np

# Get a sample datafile with a C48 cubesphere mesh.
file_path = get_data_path(["NetCDF", "unstructured_grid", "lfric_surface_mean.nc"])

with PARSE_UGRID_ON_LOAD.context():
    cube_rotatedcs = load_cube(file_path, 'rainfall_flux')
    # Simply make a separate copy with its own separate mesh (for now)
    orig_cube_copy = load_cube(file_path, 'rainfall_flux')

assert cube_rotatedcs.mesh == orig_cube_copy.mesh
assert cube_rotatedcs.mesh is not orig_cube_copy.mesh
assert cube_rotatedcs.mesh.node_coords.node_x is not orig_cube_copy.mesh.node_coords.node_x

print(cube_rotatedcs)
print(f'\nCubesphere "N"={int(np.sqrt(cube_rotatedcs.shape[-1] / 6))}')

In [ ]:
# Rotate the mesh coordinates to produce a more "interesting" mesh for our regrid target.

from iris import coord_systems as icrs

crs_plain = icrs.GeogCS(6.e6)
crs_rot = icrs.RotatedGeogCS(35.7, 0.45, ellipsoid=crs_plain)
ccrs_plain = crs_plain.as_cartopy_crs()
ccrs_rot = crs_rot.as_cartopy_crs()


# we need to grab the node locations and transform them.
cube_nodes_xco = cube_rotatedcs.mesh.node_coords.node_x
cube_nodes_yco = cube_rotatedcs.mesh.node_coords.node_y

lat_pts = cube_nodes_yco.points
lat_bds = cube_nodes_yco.bounds
lon_pts = cube_nodes_xco.points
lon_bds = cube_nodes_xco.bounds

print(f'Original lon-pts minmax = {lon_pts.min(), lon_pts.max()}')
print(f'Original lat-pts minmax = {lat_pts.min(), lat_pts.max()}')

xxx_pts = ccrs_rot.transform_points(ccrs_plain, lon_pts, lat_pts)
# xxx_bds = ccrs_rot.transform_points(ccrs_plain, lon_bds, lat_bds)
print(f'transformed, shape = {xxx_pts.shape}')


lon_pts_tx = xxx_pts[:, 0]
lat_pts_tx = xxx_pts[:, 1]
print(f'Tranformed lon-pts minmax = {lon_pts_tx.min(), lon_pts_tx.max()}')
print(f'Tranformed lat-pts minmax = {lat_pts_tx.min(), lat_pts_tx.max()}')

# Force-reset the coords
cube_nodes_xco.points = lon_pts_tx
cube_nodes_yco.points = lat_pts_tx
assert cube_rotatedcs.mesh != orig_cube_copy.mesh

In [ ]:
# Load a basic (small) UM file for its lat-lon grid.
fp2 = get_data_path(['FF', 'n48_multi_field'])
import iris.exceptions
iris.exceptions.IgnoreCubeException()

# (A bit awkward, as there are two cubes with the same phenom name : one is a mean)
def no_cellmeth(cube, field, filename):
    if cube.cell_methods:
        raise iris.exceptions.IgnoreCubeException()

latlon_cube = iris.load_cube(fp2, 'air_temperature', callback=no_cellmeth)
orog_cube = iris.load_cube(fp2, 'surface_altitude')

for coord_name in ("longitude", "latitude"):
    coord = latlon_cube.coord(coord_name)
    if not coord.has_bounds():
        coord.guess_bounds()

print(latlon_cube)

In [ ]:
# Fetch some nice-looking original LFRic (unstructured) test data.
file_path = get_data_path(["NetCDF", "unstructured_grid", "lfric_ngvat_3D_1t_full_level_face_grid_main_area_fraction_unit1.nc"])

from iris.experimental.ugrid.load import PARSE_UGRID_ON_LOAD
with PARSE_UGRID_ON_LOAD.context():
    testdata_cube = load_cube(file_path, "area_fraction")

print(testdata_cube)

In [ ]:
# Use iris-esmf-regrid for regridding
import esmf_regrid as ief
from esmf_regrid.experimental.unstructured_scheme import MeshToGridESMFRegridder
from esmf_regrid.experimental.unstructured_scheme import GridToMeshESMFRegridder

testdata_on_latlon_cube = MeshToGridESMFRegridder(testdata_cube, latlon_cube)(testdata_cube)
test_backon_c12_cube = GridToMeshESMFRegridder(testdata_on_latlon_cube, testdata_cube)(testdata_on_latlon_cube)
test_on_rotatedC48_cube = GridToMeshESMFRegridder(testdata_on_latlon_cube, cube_rotatedcs)(testdata_on_latlon_cube)

#### Visualisation steps are below this point

In [ ]:
from geovista import Transform


def pv_from_unstructcube(cube):
    lon, lat = cube.mesh.node_coords
    face_node = cube.mesh.face_node_connectivity
    # Returns a PyVista PolyData class.
    return Transform.from_unstructured(
        xs=lon.points,
        ys=lat.points,
        connectivity=face_node.indices_by_location(),
        start_index=face_node.start_index,
        data=cube.data,
        name=cube.name()
    )


def pv_from_structcube(cube):
    xco = cube.coord(axis='x')
    yco = cube.coord(axis='y')
    for co in (xco, yco):
        if not co.has_bounds():
            co.guess_bounds()
    # Returns a PyVista PolyData class.
    return Transform.from_1d(xs=xco.bounds, ys=yco.bounds, data=cube.data, name=cube.name())


# Display a single nice-looking layer
onelayer_pd = pv_from_unstructcube(testdata_cube[0,10])
testdata_on_latlon_pd = pv_from_structcube(testdata_on_latlon_cube[0,10])
test_backon_c12_pd = pv_from_unstructcube(test_backon_c12_cube[0,10])
test_on_rotatedC48_pd = pv_from_unstructcube(test_on_rotatedC48_cube[0,10])

In [ ]:
from geovista.geoplotter import GeoPlotter
from geovista import theme
import pyvista as pv
pv.global_theme.font.color = "black"
pv.global_theme.edge_color = "grey"
pv.global_theme.cmap = "blues"

my_plotter = GeoPlotter(shape=(2, 2))

polydata_names = [
    (onelayer_pd, "Original mesh data"),
    (testdata_on_latlon_pd, "Regridded onto latlon"),
    (test_backon_c12_pd, "Latlon -> back to C12"),
    (test_on_rotatedC48_pd, "Latlon -> rotated C48"),
]

for ix, pd_tuple in enumerate(polydata_names):
    my_plotter.subplot(*np.unravel_index(ix, (2, 2)))
    my_plotter.add_text(pd_tuple[-1], font_size=12)
    my_plotter.add_coastlines()
    my_plotter.add_mesh(pd_tuple[0], show_edges=True)

my_plotter.link_views()
my_plotter.camera.position = [0, -2.5, 2.5]
my_plotter.show(jupyter_backend="ipyvtklink")
# my_plotter.show(jupyter_backend="static")

---

## Projections

In [ ]:
# Load the data we'll be plotting.

file_path = get_data_path(["NetCDF", "unstructured_grid", "lfric_surface_mean.nc"])
with PARSE_UGRID_ON_LOAD.context():
    my_cube = load_cube(file_path, "zh")

# Time coord len==1 - should be a scalar coord, achieve this via indexing.
my_cube = my_cube[0]
print(my_cube)

# Convert Cube to face-based PyVista PolyData.
my_polydata = pv_from_unstructcube(my_cube)

In [ ]:
# GeoVista coastline projection not yet supported. Use a representation of coastlines as Cube data instead.

# import requests
# r = requests.get("https://github.com/SciTools-incubator/presentations/raw/main/ngms_champions_2022-04-12/coastline_grid.nc")
# open("coastline_grid.nc", "wb").write(r.content)

coastline_cube = load_cube("coastline_grid.nc")

coastline_polydata = pv_from_structcube(coastline_cube)
# Remove all NaN's (grid squares that aren't on a coast).
coastline_polydata = coastline_polydata.threshold()

In [ ]:
from geovista.crs import WGS84

def plot(plotter):
    # Add the coastline cells 'above' the data itself.
    plotter.add_mesh(
        coastline_polydata,
        color="white",
        show_edges=True,
        edge_color="white",
        radius=1.1,     # For globe plots
        zlevel=10,       # For planar plots
    )
    plot_polydata = my_polydata.copy()
    plotter.add_mesh(plot_polydata)
    if plotter.crs != WGS84:
        # Projected plot.
        plotter.camera_position = "xy"
        backend = "static"
    else:
        backend = "pythreejs"
#         backend = "static"
    plotter.show(jupyter_backend=backend)

In [ ]:
globe_plotter = GeoPlotter()
plot(globe_plotter)

#### Make projected plots using Proj strings

Remember that we're still working on coastline support for projections!

In [ ]:
moll_plotter = GeoPlotter(crs="+proj=moll")
plot(moll_plotter)

In [ ]:
moll_plotter_90 = GeoPlotter(crs="+proj=moll +lon_0=90")
plot(moll_plotter_90)

In [ ]:
eqc_plotter = GeoPlotter(crs="+proj=eqc")
plot(eqc_plotter)

---

## Any Mesh Layout

Examples from past demonstrations/experiments:

### Node-based data (LFRic orography)

![LFRic Nodes](geovista_lfric_nodes.png)

### 2D coordinates (ORCA)

![NEMO](geovista_nemo.png)

### Hexes

![Hex Grid](geovista_hex.png)

### Arbitrary face 'sidedness' (FESOM)

![FESOM](geovista_fesom.png)

---

## Unstructured Computations

### Regional extraction

In [ ]:
from geovista.geodesic import BBox

global_polydata = my_polydata

# Get those indices of the mesh that are within a region.
region = BBox(lons=[0, 70, 70, 0], lats=[-25, -25, 45, 45])
region_polydata = region.enclosed(global_polydata, preference="center")

# Note the reduced cells and points.
print(global_polydata)
print(region_polydata)

In [ ]:
# The ID's of those cells that are within the region.
indices = region_polydata["vtkOriginalCellIds"]
print(indices)

In [ ]:
from iris.experimental.ugrid import Mesh

global_cube = my_cube

# Use the derived indices to subset the original Cube.
region_cube = global_cube[indices]

# Convert the sub-setted Cube back into a Cube-with-Mesh.
new_mesh = Mesh.from_coords(*region_cube.coords(dimensions=0))
new_mesh_coords = new_mesh.to_MeshCoords(global_cube.location)
for coord in new_mesh_coords:
    region_cube.remove_coord(coord.name())
    region_cube.add_aux_coord(coord, 0)

# Note the reduced number of faces.
print(global_cube)
print(region_cube)

**Emphasis:** this workflow did not require plotting at any step.

We will now plot the result just so you can see it.

In [ ]:
def plot_region(polydata, title=None):
    region_plotter = GeoPlotter(crs="+proj=moll +lon_0=135")    
    region_plotter.add_mesh(
        coastline_polydata,
        color="black",
        show_edges=True,
        edge_color="black",
        zlevel=10
    )
    region_plotter.add_mesh(polydata)
    region_plotter.camera_position = "xy"
    if title:
        region_plotter.add_text(title, font_size=12)
    region_plotter.show(jupyter_backend="static")

plot_region(pv_from_unstructcube(region_cube))

Latitude bounds (as opposed to great circles) are also possible

### Combining meshes

### and many more...

---

## Library Interoperability

GeoVista objects are created from basic Python arrays - therefore flexible enough to work with _all_ popular scientific Python packages.

In [ ]:
import xarray

global_ds = xarray.open_dataset(get_data_path(["NetCDF", "unstructured_grid", "lfric_surface_mean.nc"]))

# Remove other dimensions for this demo - geovista only wants the horizontal dimension.
global_ds = global_ds.isel(time_counter=0)

print(global_ds)

In [ ]:
def pv_from_xarray(ds):
    # This solution is specific to this file - there isn't yet an official way to represent UGRID in xarray.
    lons = ds.variables["Mesh2d_half_levels_node_x"]
    lats = ds.variables["Mesh2d_half_levels_node_y"]
    face_node = ds.variables["Mesh2d_half_levels_face_nodes"]
    indices = face_node.data
    ds_data = ds.variables["zh"].data

    return Transform.from_unstructured(
        lons.data, lats.data, indices, data=ds_data,
        start_index=face_node.attrs["start_index"]
    )

global_polydata_xarray = pv_from_xarray(global_ds)

print(global_polydata)
print(global_polydata_xarray)

In [ ]:
# Regionally subset the polydata and use the indices to subset the original xarray.
region_polydata_xarray = region.enclosed(global_polydata_xarray, preference="center")
indices_xarray = region_polydata_xarray["vtkOriginalCellIds"]
region_ds = global_ds.isel(nMesh2d_half_levels_face=indices_xarray)

# Note the reduced number of faces.
print(global_ds)
print(region_ds)

In [ ]:
plot_region(pv_from_unstructcube(region_cube), "Via Iris")
plot_region(pv_from_xarray(region_ds), "Via xarray")